# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [7]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [4]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/Users/mahangu/Documents/code/udacity/projects/p2


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [5]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [6]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [2]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [3]:
## Keyspace creation code taken from Udacity exerices.

try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [4]:
## Keyspace set code taken from Udacity exerices.

try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## Query 1

In [5]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

query = "CREATE TABLE IF NOT EXISTS artist_song_length_by_session "
query = query + "(session_id int, session_items int, artist_name text, song_title text, song_length double, PRIMARY KEY (session_id, session_items))"
try:
    session.execute(query)
except Exception as e:
    print(e)

                    

In [8]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

# Import only the columns we need
# Via - https://www.pythonprogramming.in/how-to-read-specific-columns-of-csv-file-using-pandas.html
df_query1 = pd.read_csv(file, usecols = ['sessionId','itemInSession', 'artist', 'song', 'length'])

df_query1.dtypes

for i, row in df_query1.iterrows(): 
    query = "INSERT INTO artist_song_length_by_session(session_id, session_items, artist_name, song_title, song_length)"
    query = query + " VALUES (%s, %s, %s, %s, %s)"
    session.execute(query, (int(row['sessionId']), int(row['itemInSession']), row['artist'], row['song'], row['length']))

#### Do a SELECT to verify that the data have been inserted into each table

In [18]:
query = "select * from artist_song_length_by_session WHERE session_id = 338 and session_items = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

# Load query output directly into a pandas dataframe so we can print column names too.
df1_output = pd.DataFrame(rows)

print(df1_output)

# for row in rows:
#     print (row.artist_name, row.song_title, row.song_length)

   session_id  session_items artist_name  song_length  \
0         338              4   Faithless     495.3073   

                        song_title  
0  Music Matters (Mark Knight Dub)  


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

## Query 2

In [8]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

query = "CREATE TABLE IF NOT EXISTS artist_song_by_user "
query = query + "(user_id int, session_items int, session_id int, song_title text, artist_name text, user_first_name text, user_last_name text, PRIMARY KEY ((user_id, session_id), session_items))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [9]:
file = 'event_datafile_new.csv'

# Import only the columns we need
# Via - https://www.pythonprogramming.in/how-to-read-specific-columns-of-csv-file-using-pandas.html
df_query2 = pd.read_csv(file, usecols = ['userId', 'itemInSession', 'sessionId', 'song', 'artist', 'firstName', 'lastName'])

for i, row in df_query2.iterrows(): 
    query = "INSERT INTO artist_song_by_user(user_id, session_items, session_id, song_title, artist_name, user_first_name, user_last_name)"
    query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
    session.execute(query, (int(row['userId']), int(row['itemInSession']), int(row['sessionId']), row['song'], row['artist'], row['firstName'], row['lastName']))
    

In [17]:
query = "select * from artist_song_by_user WHERE user_id = 10 and session_id = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

# Load query output directly into a pandas dataframe so we can print column names too.

df2_output = pd.DataFrame(rows)

print(df2_output)
    
# for row in rows:
#     print (row.artist_name, row.song_title, row.session_items, row.user_first_name, row.user_last_name)
    
                    

   user_id  session_id  session_items        artist_name  \
0       10         182              0   Down To The Bone   
1       10         182              1       Three Drives   
2       10         182              2  Sebastien Tellier   
3       10         182              3      Lonnie Gordon   

                                          song_title user_first_name  \
0                                 Keep On Keepin' On          Sylvie   
1                                        Greece 2000          Sylvie   
2                                          Kilometer          Sylvie   
3  Catch You Baby (Steve Pitron & Max Sanna Radio...          Sylvie   

  user_last_name  
0           Cruz  
1           Cruz  
2           Cruz  
3           Cruz  


## Query 3

In [19]:
query = "CREATE TABLE IF NOT EXISTS user_by_song_all_hands "
query = query + "(song_title text, user_id int, user_first_name text, user_last_name text, artist_name text, PRIMARY KEY (song_title, user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

In [23]:
file = 'event_datafile_new.csv'

# Import only the columns we need
# Via - https://www.pythonprogramming.in/how-to-read-specific-columns-of-csv-file-using-pandas.html
df_query3 = pd.read_csv(file, usecols = ['song', 'userId', 'firstName', 'lastName', 'artist'])

for i, row in df_query3.iterrows(): 
    query = "INSERT INTO user_by_song_all_hands(song_title, user_id, user_first_name, user_last_name, artist_name)"
    query = query + " VALUES (%s, %s, %s, %s, %s)"
    session.execute(query, (row['song'], int(row['userId']), row['firstName'], row['lastName'], row['artist']))

In [19]:
query = "select * from user_by_song_all_hands WHERE song_title = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

# Load query output directly into a pandas dataframe so we can print column names too.

df3_output = pd.DataFrame(rows)

print(df3_output)

# for row in rows:
#     print (row.user_first_name, row.user_last_name, row.song_title)
                    

                  song_title  user_id     artist_name user_first_name  \
0  All Hands Against His Own       29  The Black Keys      Jacqueline   
1  All Hands Against His Own       80  The Black Keys           Tegan   
2  All Hands Against His Own       95  The Black Keys            Sara   

  user_last_name  
0          Lynch  
1         Levine  
2        Johnson  


### Drop the tables before closing out the sessions

In [4]:
## TO-DO: Drop the table before closing out the sessions

In [1]:
query = "drop table if exists artist_song_length_by_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table if exists artist_song_by_user"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table if exists user_by_song_all_hands"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)


name 'session' is not defined
name 'session' is not defined
name 'session' is not defined


### Close the session and cluster connection¶

In [48]:
session.shutdown()
cluster.shutdown()